### 特征工程


In [7]:
import pandas as pd


data = pd.read_csv('train.csv')
X = data.iloc[:,1:]
y = data.iloc[:,0]
X,y

(       pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
 0           0       0       0       0       0       0       0       0       0   
 1           0       0       0       0       0       0       0       0       0   
 2           0       0       0       0       0       0       0       0       0   
 3           0       0       0       0       0       0       0       0       0   
 4           0       0       0       0       0       0       0       0       0   
 ...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
 41995       0       0       0       0       0       0       0       0       0   
 41996       0       0       0       0       0       0       0       0       0   
 41997       0       0       0       0       0       0       0       0       0   
 41998       0       0       0       0       0       0       0       0       0   
 41999       0       0       0       0       0       0       0       0       0   
 
        pixel9

In [19]:
# filter 过滤法
# 方差过滤法

from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
# 可以通过参数控制方差小于多少不要
selector = VarianceThreshold(X.var().median())
# 删除方差为0的特征
X_var0 = selector.fit_transform(X)
# 减少了70多个特征
X_var0.shape,X.var().median()

cross_val_score(RFC(), X_var0, y, cv=5).mean()




np.float64(0.9646666666666667)

In [ ]:
# 卡方过滤法, 相关性过滤， 原假设，特征之间相互独立
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import cross_val_score

# 选择k个最好的特征
# 这里的k是特征的个数
X_fschi = SelectKBest(chi2, k=300).fit_transform(X_var0, y)
# cross_val_score(RFC(), X_fschi, y, cv=5).mean()
# p <= 0.05 相关 > 0.05 独立
chivalue,p = chi2(X, y)
p.shape

(784,)

In [ ]:
# F 检验 ， 两组数据的线性关系，原假设，数据不存在显著线性关系
from sklearn.feature_selection import SelectKBest, f_classif
# p <= 0.05 不成立 > 0.05 成立
F,pvalues_f = f_classif(X_var0,y)
pvalues_f

array([0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 4.71193533e-220,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0

In [37]:
# 互信息法 0 独立，1相关

from sklearn.feature_selection import SelectKBest, mutual_info_classif

r = mutual_info_classif(X_var0, y, discrete_features=True)
r


array([0.07953354, 0.09939235, 0.11661064, 0.12920102, 0.13230057,
       0.12094792, 0.09888406, 0.07210726, 0.09729366, 0.1213095 ,
       0.1429877 , 0.16468031, 0.18103441, 0.18878465, 0.17731905,
       0.15133659, 0.11694939, 0.08772453, 0.06232601, 0.04742964,
       0.09160918, 0.12349804, 0.15214587, 0.17839667, 0.20512801,
       0.23272531, 0.25296523, 0.26145042, 0.24408941, 0.20642641,
       0.16582918, 0.12821809, 0.10153175, 0.08410604, 0.07196919,
       0.05714531, 0.0810894 , 0.10718069, 0.13007657, 0.15041966,
       0.16417111, 0.17645624, 0.19119513, 0.2099301 , 0.21449252,
       0.20133986, 0.18082971, 0.1599463 , 0.14014245, 0.12381234,
       0.11112004, 0.09671726, 0.08760297, 0.07028094, 0.08936148,
       0.10287435, 0.11490536, 0.12304073, 0.13300834, 0.13658985,
       0.13587705, 0.14354307, 0.14941074, 0.1460547 , 0.13833213,
       0.14246277, 0.14342419, 0.13596286, 0.12208445, 0.10845194,
       0.09597673, 0.06139883, 0.07995161, 0.0918733 , 0.10239

In [ ]:
# embedded 嵌入法 ，有coef_, importans 或者 l1 ，l2 都可以使用
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score

rfc = RFC(n_estimators=10, random_state=0)

X_embedded = SelectFromModel(rfc, threshold="median").fit_transform(X, y)

X_embedded.shape

(42000, 392)

In [43]:
# 包装法 , 逻辑回归优先使用嵌入法，svm 优先使用包装法
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import cross_val_score

rfc = RFC(n_estimators=10, random_state=0)
rfe = RFE(rfc, n_features_to_select=51)
X_rfe = rfe.fit_transform(X, y)
rfe.ranking_,rfe.support_.sum()

(array([730, 726, 725, 724, 723, 721, 719, 716, 715, 713, 711, 708, 706,
        701, 700, 734, 697, 692, 690, 687, 685, 684, 683, 681, 718, 678,
        677, 696, 710, 668, 667, 666, 707, 702, 664, 662, 661, 722, 660,
        659, 658, 691, 657, 686, 652, 651, 644, 642, 641, 729, 682, 640,
        639, 672, 636, 705, 634, 693, 633, 632, 714, 698, 649, 732, 635,
        620, 599, 539, 456, 416, 403, 431, 436, 421, 448, 596, 530, 591,
        604, 638, 637, 619, 703, 617, 622, 600, 704, 597, 653, 582, 645,
        517, 388, 387, 370, 321, 365, 261, 274, 293, 264, 249, 258, 359,
        338, 374, 412, 518, 615, 631, 618, 609, 614, 616, 602, 570, 621,
        613, 474, 399, 360, 320, 279, 248, 170, 227, 141, 192, 210, 237,
        269, 283, 303, 340, 386, 393, 489, 583, 595, 605, 586, 584, 581,
        569, 603, 415, 384, 343, 311, 209, 148,  23,  11,   1,   1,   1,
         73,   1, 118, 153, 226, 266, 301, 353, 401, 514, 579, 563, 536,
        541, 540, 535, 424, 373, 330, 267, 211, 126